In [1]:
!pip install -q langchain langchainhub langchain-community langchain-core langchain-groq pypdf docx2txt python-docx pandas beautifulsoup4 markdown sentence-transformers faiss-cpu tiktoken



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [2]:
import langchain
print(langchain.__version__)

0.3.26


In [ ]:
import os

os.environ["GROQ_API_KEY"] = ""


In [17]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",  # or "llama3-70b-8192"
    temperature=0.7,
)

llm_response = llm.invoke("Tell me a joke")

llm_response

AIMessage(content="Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(wait for it...)\n\nBecause it was two-tired!\n\nHope that made you laugh!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 14, 'total_tokens': 46, 'completion_time': 0.031843546, 'prompt_time': 0.00398632, 'queue_time': 0.164348829, 'total_time': 0.035829866}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_8dc6ecaf8e', 'finish_reason': 'stop', 'logprobs': None}, id='run--574390bf-7ea4-46c4-8582-6d3532a56fcb-0', usage_metadata={'input_tokens': 14, 'output_tokens': 32, 'total_tokens': 46})

In [18]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(llm_response)

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(wait for it...)\n\nBecause it was two-tired!\n\nHope that made you laugh!"

In [19]:
chain = llm | output_parser
chain.invoke("Tell me a joke")

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(wait for it...)\n\nBecause it was two-tired!\n\nHope that made you laugh!"

In [20]:
from typing import List
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
    phone_model: str = Field(description="Name and model of the phone")
    rating: float = Field(description="Overall rating out of 5")
    pros: List[str] = Field(description="List of positive aspects")
    cons: List[str] = Field(description="List of negative aspects")
    summary: str = Field(description="Brief summary of the review")

review_text = """
Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
stronger. Battery life's solid, lasts me all day no problem.

Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
Also, still getting used to the new button layout, keep hitting Bixby by mistake.

Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
being perfect. If you're due for an upgrade, definitely worth checking out!
"""

structured_llm = llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)
output

MobileReview(phone_model='Galaxy S21', rating=4.0, pros=['gorgeous screen', 'insane camera'], cons=['expensive', 'no charger included', 'annoying button layout'], summary='Solid phone with a few quirks')

In [21]:
output.pros

['gorgeous screen', 'insane camera']

In [22]:
output.cons

['expensive', 'no charger included', 'annoying button layout']

In [23]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
prompt.invoke({"topic": "programmer"})

ChatPromptValue(messages=[HumanMessage(content='Tell me a short joke about programmer', additional_kwargs={}, response_metadata={})])

In [24]:
chain = prompt | llm | output_parser
chain.invoke({"topic":"programmer"})

'Why do programmers prefer dark mode?\n\nBecause light attracts bugs.'

In [30]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Define the prompt
prompt = ChatPromptTemplate.from_template(" Help the farmer {topic}")

# Initialize the LLM model
llm = ChatGroq(
    model="llama3-8b-8192",  # or "llama3-70b-8192"
    temperature=0.7,
)

# Define the output parser
output_parser = StrOutputParser()

# Define the chain
chain = prompt | llm | output_parser

result = chain.invoke({"topic":"What is the different type of pestiside are needed when paddy is attacking some kind of insect which is very dangerous?"})
print(result)


When paddy is attacked by a severe pest or insect infestation, the farmer may need to use a combination of pest control methods, including pesticides. Here are some common types of pesticides that can be used to control pests that are commonly found in paddy fields:

1. **Insecticides**:
	* Organophosphates (e.g., Chlorpyrifos, Malathion): effective against a wide range of pests, including rice stem borers, leaf folders, and planthoppers.
	* Pyrethroids (e.g., Cyhalothrin, Deltamethrin): effective against pests like rice whorl maggots, leafhoppers, and grasshoppers.
	* Neonicotinoids (e.g., Imidacloprid, Thiamethoxam): effective against pests like rice leaf folders, planthoppers, and stem borers.
2. **Herbicides**:
	* Selective herbicides (e.g., Glyphosate, 2,4-D): used to control weeds that compete with the paddy crop for water, nutrients, and light.
3. **Fungicides**:
	* Carboxin: effective against fungal diseases like rice blast and brown spot.
	* Prochloraz: effective against funga

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(content="You are a helpful assistant that tells jokes.")
human_message = HumanMessage(content="Tell me about programming")
llm.invoke([system_message, human_message])

AIMessage(content="Programming! It's like trying to solve a puzzle, except the puzzle is trying to solve you back!\n\nBut seriously, programming is like building with Legos, except instead of blocks, you're using code to create something amazing. And just like Legos, it can be frustrating when things don't fit together quite right, but the end result is always worth it!\n\nHere's a joke to help you debug your code:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs!\n\nAnd here's another one:\n\nWhy do programmers prefer coffee?\n\nBecause it's a bug-free drink!\n\nOkay, okay, I'll stop with the puns now. But programming is really all about creativity and problem-solving. With the right tools and mindset, you can create something truly amazing!\n\nSo, what do you want to build next?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 166, 'prompt_tokens': 28, 'total_tokens': 194, 'completion_time': 0.184370497, 'prompt_time': 0.005201038,

In [27]:
template = ChatPromptTemplate([
    ("system", "You are a helpful assistant that tells jokes."),
    ("human", "Tell me about {topic}")
])

prompt_value = template.invoke(
    {
        "topic": "programming"
    }
)
prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant that tells jokes.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about programming', additional_kwargs={}, response_metadata={})])

In [28]:
llm.invoke(prompt_value)

AIMessage(content='Programming! It\'s like trying to teach a computer to do a backflip... but instead of a backflip, it\'s just a bunch of code that makes a website work. \n\nBut seriously, programming is like being a master builder, constructing entire worlds (or at least, really cool websites and apps) with just a few lines of code. And just like how you need the right tools and materials to build a house, programmers need the right programming languages and tools to create their digital masterpieces.\n\nHere\'s a joke to help you "debug" your understanding of programming:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs!\n\nHope that made you LOL! Do you have any specific questions about programming or would you like to hear another joke?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 152, 'prompt_tokens': 28, 'total_tokens': 180, 'completion_time': 0.168612086, 'prompt_time': 0.004889884, 'queue_time': 0.16941715, 'total_time':